In [ ]:
import numpy as np 
from scipy.special import k1
import pandas as pd 
from scipy.stats import norminvgauss 
from tqdm import tqdm 

def sample_nig(num , alpha , beta , gamma , mu ): 
    return norminvgauss.rvs(alpha*gamma , beta*gamma , mu , gamma , size = num  ) 

def pdf_nig(x , alpha, beta , gamma , mu ): 
    return norminvgauss.pdf(x  , alpha*gamma  , beta*gamma , mu , gamma  )

def pdf_nig_man(x , alpha , beta , gamma , mu ): 
    lambda_ = np.sqrt(alpha ** 2 - beta ** 2 ) 
    part1 = alpha * gamma * k1(alpha * np.sqrt( gamma ** 2 + (x- mu)**2))
    part2 = np.pi * np.sqrt(gamma**2 + (x-mu) ** 2 )
    part3 = np.exp(gamma * lambda_ + beta* (x- mu  )) 
    return (part1/ part2 ) * part3 
print(pdf_nig(0.1 , 2 , 0.2 , 0.8 , 0.04 ))
print(pdf_nig_man(0.1 , 2 , 0.2 , 0.8 , 0.04 ))
import numpy as np
from scipy.special import kv  # modified Bessel Kν

def p_prime(x, alpha, beta, gamma, mu):
    delta = gamma                   # follow the user's naming
    t = x - mu
    R = np.sqrt(delta**2 + t**2)
    z = alpha * R

    K0 = kv(0, z)
    K1 = kv(1, z)
    K2 = kv(2, z)

    big_gamma = np.sqrt(alpha**2 - beta**2)   # γ = √(α²−β²)
    C = (alpha * delta / np.pi) * np.exp(delta * big_gamma + beta * t)

    term1 = beta * K1 / R
    term2 = t / R**3 * (-(alpha * R * (K0 + K2) / 2) - K1)

    return C * (term1 + term2)


def payoff_fn(x , K ): 
    return 50 * np.max( np.exp(x) - K  , 0 ) 
def H1(x , K , theta , alpha , beta , gamma , mu ): 
    part1 = np.exp(-2 * np.abs(theta)) 
    part2 = payoff_fn(x , K )**2 
    part3 = p_prime(x - 2*theta , alpha , beta , gamma , mu ) / pdf_nig(x , alpha , beta , gamma , mu ) 
    part4 = (pdf_nig(x - theta , alpha , beta , gamma , mu   ) / pdf_nig(x - 2*theta , alpha , beta , gamma , mu ))**2 
    return part1 * part2 * part3 * part4 

def phi(theta , alpha , beta , gamma , mu ): 
    return mu*theta + gamma * (np.sqrt(alpha ** 2 - beta ** 2 ) - np.sqrt(alpha ** 2  - (beta + theta )**2 )) 
def phi_prime(theta, alpha, beta, gamma, mu):
    return mu + gamma * (beta + theta) / np.sqrt(alpha**2 - (beta + theta)**2)

def T(theta, alpha , beta ): 
    return ( beta - alpha )* theta / np.sqrt(1 + theta**2 )

def T_prime(theta , alpha , beta ): 
    return (beta - alpha )/ (np.sqrt(1 +theta**2 ) ** 3 )
def H2( x , K , theta , alpha , beta , gamma , mu ): 
    part1 = np.exp(-np.abs(theta )) 
    part2 = payoff_fn(x , K)**2 
    part3 = (phi_prime(theta , alpha , beta , gamma , mu ) - x )

    return part1 * part2 * part3

    
def get_theta_translation(theta_0 , K , num_iter , alpha , beta , gamma , mu ): 
    samples = sample_nig(num_iter , alpha , beta , gamma , mu ) 
    for i in tqdm(range(num_iter)): # on ajout le calcul par T pour stabilise 
        #theta = T(theta_0 , alpha , beta )
        step = 1 / (1000 + i )
        #theta_0 -= step * T_prime(theta_0 , alpha, beta ) *  H1(samples[i] , K , theta, alpha , beta , gamma, mu )  
        theta_0 -= step *   H1(samples[i] , K , theta_0, alpha , beta , gamma, mu )  

    return T(theta_0 , alpha , beta ) 
def get_theta_escher(theta_0 , K , num_iter , alpha , beta , gamma , mu ): 
    for i in tqdm(range(num_iter)) : 
        theta = T(theta_0 , alpha, beta )
        sample = sample_nig(1 , alpha , beta - theta , gamma , mu )[0] 
        step = 1 / (1000 + i ) 
        theta_0 -= step * T_prime(theta_0 , alpha , beta )* H2(sample , K , theta , alpha , beta , gamma , mu )

    return T(theta_0  , alpha , beta ) 
def simulate_mc_crude(num_iter , strike  , alpha , beta , gamma , mu ) :
    value_0 = 0 
    samples = sample_nig(num_iter , alpha , beta , gamma , mu )
    for i in range(num_iter): 
        value_0 += payoff_fn(samples[i] , strike) 

    return value_0/num_iter  


In [29]:
simulate_mc_crude( 10000 , 1 , 2 , 0.2 , 0.8 , 0.04 )

20.05257342002527

In [30]:
thet_escher = get_theta_escher(0.3 , 1 , 100000 , 2 , 0.2 , 0.8 , 0.04 )

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100000/100000 [00:07<00:00, 12565.33it/s]


In [31]:
thet_escher

1.79298962248385

In [32]:
thet_trans = get_theta_translation(0.3, 1 , 100000 , 2 , 0.2 , 0.8 , 0.04 )

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100000/100000 [00:19<00:00, 5150.68it/s]


In [33]:
thet_trans

-1.7992567651863451

In [34]:
def MC_translation(M  , K , theta_opt , alpha , beta , gamma , mu ) :  
    val_ = 0 

    for i in tqdm(range(M)  ) : 
        sample = sample_nig(K , alpha , beta , gamma , mu )[0] 
        val_ += payoff_fn(sample , K ) *  pdf_nig(sample  + theta_opt , alpha , beta , gamma , mu ) / pdf_nig(sample  + theta_opt , alpha , beta , gamma , mu )

    return val_/M

In [35]:
price = MC_translation(100000 , 1 , thet_trans , 2 , 0.2  , 0.8 , 0.04 )

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100000/100000 [00:19<00:00, 5176.57it/s]


In [36]:
price

20.834591772974896

In [37]:
def MC_esher(M , K , theta_opt , alpha , beta , gamma , mu ) : 
    val_ = 0 

    for i in tqdm(range(M)) : 
        sample = sample_nig(1 , alpha , beta + theta_opt , gamma , mu  )[0] 
        val_ += payoff_fn(sample , K )* np.exp(- theta_opt * sample)
    mean = val_ 
    mean = mean * np.exp(phi(theta_opt , alpha , beta , gamma , mu )) /M 

    return mean 

In [39]:
def MC_esscher(M, K, theta_opt, alpha, beta, gamma, mu):
    assert abs(beta + theta_opt) < alpha, "Esscher θ outside NIG CGF domain"

    total = 0.0
    for _ in tqdm(range(M)):
        x = np.squeeze(sample_nig(1, alpha, beta + theta_opt, gamma, mu))
        total += payoff_fn(x, K) * np.exp(-theta_opt * x)

    estimate = (total / M) * np.exp(phi(theta_opt, alpha, beta, gamma, mu))
    return estimate

In [40]:
price_escher = MC_esher(100000 , 1, thet_escher , 2 , 0.2 , 0.8 , 0.04 )

 46%|██████████████████████████████████████████████████████████████████▍                                                                            | 46420/100000 [00:03<00:03, 13796.92it/s]/var/folders/hl/wrf9xwx508315bwwmp8vgxp80000gn/T/ipykernel_26327/3373073489.py:2: RuntimeWarning: overflow encountered in exp
  return 50 * np.max( np.exp(x) - K  , 0 )
/var/folders/hl/wrf9xwx508315bwwmp8vgxp80000gn/T/ipykernel_26327/4004288728.py:6: RuntimeWarning: invalid value encountered in scalar multiply
  val_ += payoff_fn(sample , K )* np.exp(- theta_opt * sample)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100000/100000 [00:07<00:00, 13616.01it/s]


In [41]:
price_escher

nan